In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib

from pathlib import Path
from pymatgen.core import Structure, Composition, Element

# user-friendly print
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [19]:
icsd_inorganic_info = pd.read_pickle("common_data/icsd_inorganic_info_20230522.pd.xz").dropna(
    subset=["full_formula", "structured_formula"]
)
icsd_inorganic_info = icsd_inorganic_info.assign(composition=icsd_inorganic_info["full_formula"].apply(Composition))
icsd_inorganic_info = icsd_inorganic_info.assign(
    reduced_composition=icsd_inorganic_info["composition"].apply(lambda s: s.reduced_composition)
)

icsd_inorganic_info.head(3)
icsd_inorganic_info.shape

/data/.miniforge3/envs/xepy311/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/data/.miniforge3/envs/xepy311/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/data/.miniforge3/envs/xepy311/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


,collection_code,full_formula,space_group_number,is_ordered,ratio,space_group,structured_formula,chemical_name,cell_volume,formula_weight,...,pressure,ab_formula,calculated_density,measured_density,quality,publication_year,journal,reference,composition,reduced_composition
idnum,,,,,,,,,,,,,,,,,,,,,
1,1,Cr4 Te8 O22,14.0,True,"(4.0, 8.0, 22.0)",P2_1/c,Cr2 Te4 O11,Dichromium(III) undecaoxo-tetratellurate(IV),507.61,790.389,...,0.101325,A2B4C11,5.17,NaN,10,1976,"Acta Crystallographica, Section B: Structural ...","Acta Crystallographica, Section B: Structural ...","(Cr, Te, O)","(Cr, Te, O)"
2,5,Na4 P4 H16 O20,33.0,True,"(4.0, 4.0, 16.0, 20.0)",Pna2_1,Na (H2 P O4) (H2 O),Sodium dihydrogenphosphate(V) hydrate,444.09,137.987,...,0.101325,ABC4D5,2.06,2.055,10,1976,"Acta Crystallographica, Section B: Structural ...","Acta Crystallographica, Section B: Structural ...","(Na, P, H, O)","(Na, P, H, O)"
3,6,Al4 Te2 S2 O18,11.0,True,"(2.0, 2.0, 4.0, 18.0)",P2_1/m,Al2 (O H)2 (Te O3) (S O4),Dialuminium dihydroxide tellurate(IV) sulfate(VI),326.11,359.631,...,0.101325,ABC2D9,3.66,3.7,-1,1976,"Acta Crystallographica, Section B: Structural ...","Acta Crystallographica, Section B: Structural ...","(Al, Te, S, O)","(Al, Te, S, O)"


(110581, 21)

In [18]:
qc_ac_dataset = pd.read_csv("common_data/composition_dataset.csv")
qc_ac_dataset = qc_ac_dataset.assign(composition=qc_ac_dataset["Original formula"].apply(Composition))
qc_ac_dataset = qc_ac_dataset.assign(
    reduced_composition=qc_ac_dataset["composition"].apply(lambda s: s.reduced_composition)
)

qc_ac_dataset.head(3)
qc_ac_dataset.shape

,ID,Combination of elements,Original formula,Alphabetical formula,Number of elements,Composition type,"QC, AC type",Stability,Cluster type,Lattice type,...,Heat treatment,Heat treatment condition,Additional fabrication information,Solidification,Method,Note,Data source,Source type,composition,reduced_composition
0,cmp0001,AgAlEr,Ag28.6Al37.1Er14.3,Ag28.6 Al37.1 Er14.3,3.0,NaN,IQC,Metastable,Tsai,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.1039/c4ra07980a,Reference,"(Ag, Al, Er)","(Ag, Al, Er)"
1,cmp0002,AgAlLu,Ag28.6Al37.1Lu14.3,Ag28.6 Al37.1 Lu14.3,3.0,NaN,IQC,Metastable,Tsai,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.1039/c4ra07980a,Reference,"(Ag, Al, Lu)","(Ag, Al, Lu)"
2,cmp0003,AgAlMg,Mg70Al20Ag10,Ag10 Al20 Mg70,3.0,Alloy,IQC,NaN,NaN,NaN,...,NaN,NaN,NaN,Melt spnning,NaN,NaN,10.1088/0953-8984/2/28/006,Main,"(Mg, Al, Ag)","(Mg, Al, Ag)"


(1448, 28)

In [36]:
matched_comps = icsd_inorganic_info[
    icsd_inorganic_info.reduced_composition.isin(qc_ac_dataset.reduced_composition)
].drop_duplicates(subset="reduced_composition")

matched_comps.composition.nunique()
matched_comps

24

,collection_code,full_formula,space_group_number,is_ordered,ratio,space_group,structured_formula,chemical_name,cell_volume,formula_weight,...,pressure,ab_formula,calculated_density,measured_density,quality,publication_year,journal,reference,composition,reduced_composition
idnum,,,,,,,,,,,,,,,,,,,,,
76776,14374,Mn24 Al114,200.0,True,"(24.0, 114.0)",Pm-3,Al57 Mn12,Aluminium manganese (57/12),2038.72,2197.1400,...,0.101325,A4B19,3.58,3.62,-1,1966,Acta Crystallographica,"Acta Crystallographica (1966) 20, p. 614-617","(Mn, Al)","(Mn, Al)"
14352,30207,Al20 Co8,194.0,True,"(8.0, 20.0)",P6_3/mmc,Al5 Co2,Aluminium cobalt (5/2),382.69,252.7600,...,0.101325,A2B5,4.39,4.14,-1,1938,Zeitschrift fuer Kristallographie,"Zeitschrift fuer Kristallographie (1938) 99, ...","(Al, Co)","(Al, Co)"
84192,57652,Al45 Cr7,12.0,True,"(7.0, 45.0)",C2/m,Al45 Cr7,Aluminum chromium (45/7),1630.21,1578.1000,...,0.101325,A7B45,3.21,3.18,-1,1960,Acta Crystallographica,"Acta Crystallographica (1960) 13, p. 257-263","(Al, Cr)","(Al, Cr)"
70979,57973,Mn2 Al12,63.0,True,"(2.0, 12.0)",Cmcm,Al6 Mn,Aluminium manganese (6/1),435.67,216.8200,...,0.101325,AB6,3.31,3.27,10,1981,"Acta Crystallographica, Section B: Structural ...","Acta Crystallographica, Section B: Structural ...","(Mn, Al)","(Mn, Al)"
71107,58110,Al13 Os4,12.0,True,"(4.0, 13.0)",C2/m,Al13 Os4,Aluminium osmium (13/4),525.41,1111.5400,...,0.101325,A4B13,7.03,NaN,10,1964,Acta Chemica Scandinavica,"Acta Chemica Scandinavica (1964) 18, p. 2294...","(Al, Os)","(Al, Os)"
71149,58153,Al20 Rh8,194.0,True,"(8.0, 20.0)",P6_3/mmc,Al5 Rh2,Aluminium rhodium (5/2),423.75,337.0490,...,0.101325,A2B5,5.28,5.3,-1,1967,Acta Chemica Scandinavica,"Acta Chemica Scandinavica (1967) 21, p. 647-651","(Al, Rh)","(Al, Rh)"
71154,58158,Al39 Ru12,12.0,True,"(12.0, 39.0)",C2/m,Al13 Ru4,Aluminium ruthenium (13/4),1575.21,755.0200,...,0.101325,A4B13,4.78,NaN,10,1965,Acta Chemica Scandinavica,"Acta Chemica Scandinavica (1965) 19, p. 2124...","(Al, Ru)","(Al, Ru)"
76020,58735,Be68 Ru12,204.0,True,"(12.0, 68.0)",Im-3,Be17 Ru3,Beryllium ruthenium (17/3),1457.12,456.4140,...,0.101325,A3B17,4.16,NaN,10,1962,Acta Crystallographica,"Acta Crystallographica (1962) 15, p. 1191-1195","(Be, Ru)","(Be, Ru)"
20975,61039,Mg4 B8 Ru10,55.0,True,"(4.0, 8.0, 10.0)",Pbam,Mg2 Ru5 B4,Dimagnesium pentaruthenium tetraboride,247.44,597.2140,...,0.101325,A2B4C5,8.02,8.015,10,1985,Zeitschrift fuer Anorganische und Allgemeine C...,Zeitschrift fuer Anorganische und Allgemeine C...,"(Mg, B, Ru)","(Mg, B, Ru)"


In [48]:
matched_structure[
    ["Combination of elements", "Original formula", "Alphabetical formula", "QC, AC type", "idnum"]
].rename(columns={"idnum": "ICSD ID"})

,Combination of elements,Original formula,Alphabetical formula,"QC, AC type",ICSD ID
0,AlRh,Al5Rh2,Al5 Rh2,DAC,71149
1,AlCo,Al5Co2,Al5 Co2,DAC,14352
2,AlRh,Al13Rh4,Al13 Rh4,DAC,116210
3,AlRu,Al13Ru4,Al13 Ru4,DAC,71154
4,AlOs,Al13Os4,Al13 Os4,DAC,71107
5,AlCo,Al13Co4,Al13 Co4,DAC,79766
6,AlFe,Al13Fe4,Al13 Fe4,DAC,210378
7,AlReSi,Al102Re24Si12,Al102 Re24 Si12,IAC,72371
8,AlCr,Al45Cr7,Al45 Cr7,IAC,84192
9,BeRu,Ru3Be17,Be17 Ru3,IAC,76020


In [44]:
matched_qc_ac = (
    qc_ac_dataset[qc_ac_dataset.reduced_composition.isin(matched_comps.reduced_composition)][
        ["Combination of elements", "Original formula", "Alphabetical formula", "reduced_composition", "QC, AC type"]
    ]
    .drop_duplicates(subset="reduced_composition")
    .sort_values("reduced_composition")
    .reset_index()
)

matched_qc_ac.shape
matched_qc_ac

(24, 6)

,index,Combination of elements,Original formula,Alphabetical formula,reduced_composition,"QC, AC type"
0,797,AlRh,Al5Rh2,Al5 Rh2,"(Al, Rh)",DAC
1,149,AlCo,Al5Co2,Al5 Co2,"(Al, Co)",DAC
2,795,AlRh,Al13Rh4,Al13 Rh4,"(Al, Rh)",DAC
3,804,AlRu,Al13Ru4,Al13 Ru4,"(Al, Ru)",DAC
4,729,AlOs,Al13Os4,Al13 Os4,"(Al, Os)",DAC
5,147,AlCo,Al13Co4,Al13 Co4,"(Al, Co)",DAC
6,466,AlFe,Al13Fe4,Al13 Fe4,"(Al, Fe)",DAC
7,790,AlReSi,Al102Re24Si12,Al102 Re24 Si12,"(Al, Re, Si)",IAC
8,240,AlCr,Al45Cr7,Al45 Cr7,"(Al, Cr)",IAC
9,891,BeRu,Ru3Be17,Be17 Ru3,"(Ru, Be)",IAC


load structures

In [26]:
icsd_inorganic_structures = pd.read_pickle("common_data/icsd_inorganic_ordered_structure_20230522.pd.xz")

icsd_inorganic_structures.head(3)
icsd_inorganic_structures.shape

,structure,full_formula,composition,is_ordered,space_group_num,space_group,wy_pattern,wy_hist,sg_wy,ratio,...,structured_formula,chemical_name,cell_volume,formula_weight,temperature,pressure,ab_formula,calculated_density,quality,publication_year
idnum,,,,,,,,,,,,,,,,,,,,,
1,"{'@module': 'pymatgen.core.structure', '@class...",Cr4 Te8 O22,"{'Cr3+': 4.0, 'Te4+': 8.0, 'O2-': 22.0}",True,14.0,P2_1/c,"{'O': ['a', 'e', 'e', 'e', 'e', 'e'], 'Cr': ['...","{'a': 0.1111111111111111, 'e': 0.8888888888888...",14_a-2_e-4_e-4_e-4_e-4_e-4_e-4_e-4_e-4,"(4.0, 8.0, 22.0)",...,Cr2 Te4 O11,Dichromium(III) undecaoxo-tetratellurate(IV),507.61,790.389,293.0,0.101325,A2B4C11,5.17,10,1976
2,"{'@module': 'pymatgen.core.structure', '@class...",Na4 P4 H16 O20,"{'Na+': 4.0, 'P5+': 4.0, 'H+': 16.0, 'O2-': 20.0}",True,33.0,Pna2_1,"{'H': ['a', 'a', 'a', 'a'], 'O': ['a', 'a', 'a...",{'a': 1.0},33_a-4_a-4_a-4_a-4_a-4_a-4_a-4_a-4_a-4_a-4_a-4,"(4.0, 4.0, 16.0, 20.0)",...,Na (H2 P O4) (H2 O),Sodium dihydrogenphosphate(V) hydrate,444.09,137.987,293.0,0.101325,ABC4D5,2.06,10,1976
3,"{'@module': 'pymatgen.core.structure', '@class...",Al4 Te2 S2 O18,"{'Al3+': 4.0, 'Te4+': 2.0, 'S6+': 2.0, 'O2-': ...",True,11.0,P2_1/m,"{'O': ['e', 'e', 'e', 'f', 'f', 'f'], 'S': ['e...","{'e': 0.5555555555555556, 'f': 0.4444444444444...",11_e-2_e-2_e-2_e-2_e-2_f-4_f-4_f-4_f-4,"(2.0, 2.0, 4.0, 18.0)",...,Al2 (O H)2 (Te O3) (S O4),Dialuminium dihydroxide tellurate(IV) sulfate(VI),326.11,359.631,293.0,0.101325,ABC2D9,3.66,-1,1976


(110581, 21)

In [45]:
matched_structure = icsd_inorganic_structures.loc[matched_comps.index]
matched_structure = matched_structure.assign(
    structure=matched_structure.structure.apply(Structure.from_dict),
)
matched_structure = (
    pd.concat([matched_structure, matched_comps], axis=1).sort_values("reduced_composition").reset_index()
)
matched_structure = pd.concat([matched_structure.drop(columns="reduced_composition"), matched_qc_ac], axis=1)

matched_structure.head(3)

,idnum,structure,full_formula,composition,is_ordered,space_group_num,space_group,wy_pattern,wy_hist,sg_wy,...,publication_year,journal,reference,composition,index,Combination of elements,Original formula,Alphabetical formula,reduced_composition,"QC, AC type"
0,71149,"[[0. 0. 3.927] Al0+, [0. 0. 0.] Al0+, [ ...",Al20 Rh8,"{'Al0+': 20.0, 'Rh0+': 8.0}",True,194.0,P6_3/mmc,"{'Al': ['a', 'h', 'k'], 'Rh': ['c', 'h']}","{'a': 0.2, 'c': 0.2, 'h': 0.4, 'k': 0.2}",194_a-2_c-2_h-6_h-6_k-12,...,1967,Acta Chemica Scandinavica,"Acta Chemica Scandinavica (1967) 21, p. 647-651","(Al, Rh)",797,AlRh,Al5Rh2,Al5 Rh2,"(Al, Rh)",DAC
1,14352,"[[0. 0. 3.7695] Al0+, [0. 0. 0.] Al0+,...",Al20 Co8,"{'Al0+': 20.0, 'Co0+': 8.0}",True,194.0,P6_3/mmc,"{'Al': ['a', 'h', 'k'], 'Co': ['c', 'h']}","{'a': 0.2, 'c': 0.2, 'h': 0.4, 'k': 0.2}",194_a-2_c-2_h-6_h-6_k-12,...,1938,Zeitschrift fuer Kristallographie,"Zeitschrift fuer Kristallographie (1938) 99, ...","(Al, Co)",149,AlCo,Al5Co2,Al5 Co2,"(Al, Co)",DAC
2,116210,"[[ 4.025 0.38419689 -2.1192818 ] Al0+, [...",Al39 Rh12,"{'Al0+': 39.0, 'Rh0+': 12.0}",True,12.0,C2/m,"{'Al': ['a', 'h', 'i', 'i', 'i', 'i', 'i', 'i'...","{'a': 0.05, 'h': 0.05, 'i': 0.6, 'j': 0.3}",12_a-2_h-4_i-4_i-4_i-4_i-4_i-4_i-4_i-4_i-4_i-4...,...,1983,Journal of the Less-Common Metals,"Journal of the Less-Common Metals (1983) 91, (...","(Al, Rh)",795,AlRh,Al13Rh4,Al13 Rh4,"(Al, Rh)",DAC


In [46]:
for i, data in matched_structure.iterrows():
    _ = data.structure.to(
        fmt="cif", filename=f"matched_structures/{data['Original formula']}_{data['QC, AC type']}.cif", symprec=0.01
    )